In [1]:
import pandas as pd
import cpca, re
from datetime import datetime

In [2]:
read_csv_path = './hunter-info.csv'
save_csv_path = './result1-2.csv'

frame_data = pd.read_csv(read_csv_path, encoding='GBK')

In [3]:
frame_data.head(3)

,hunter_id,hunter_name,hunter_sex,hunter_bthday,hunter_addr,hunter_exp,hunter_soci,hunter_eval,exp_position,exp_min_wage,...,exp_report,resume_keys,job_exps,project_exps,competition_exps,education_exps,training_exps,skill_exps,language_exps,cert_exps
0,1649711597649788928,刘女士,1.0,2002-04-16T16:00:00.000Z,湖北省武汉市武昌区,1年工作经验,LEAGUE_MEMBER,统计专业学生，熟悉SPSS软件、pyhton、mysql软件的基本操作；曾经在中国铁塔公司贵...,"['数据分析师', '其他']",3000,...,时间待议,"['统计专业', 'spss', 'mysql', 'python']",['中国铁塔公司贵港分公司[档案部实习生]:负责档案数据的录入与整理'],[],[],['湖北大学[本科]:应用统计学'],[],"['python[SKILLED]', 'mysql[SKILLED]', 'R语言[SKI...",[],[]
1,1649221278801985536,王先生,0.0,1999-11-16T16:00:00.000Z,广东省广州市白云区,无经验,LEAGUE_MEMBER,1.曾经在学校社团里面，当人副部长，在社团进行大型对内外活动的时候，组织进行游戏，规划每个摊...,"['数据分析师', '数据挖掘工程师', '其他']",5000,...,随时到岗,[],[],[],[],[],[],[],[],[]
2,1648221000086716416,特先生,0.0,2023-04-03T16:00:00.000Z,北京市北京市西城区,1年工作经验,LEAGUE_MEMBER,awdaw,"['数据挖掘工程师', '图像处理工程师']",4000,...,1周后到岗,"['潮男', '靓仔']",['中国石油大[教授]:dawdawdasxZxxerg'],['dawda[daw]:daxzxwefr'],['泰迪杯[特等奖]:第七届泰迪杯数据挖掘挑战赛'],['中国石油大学[硕士]:daw'],['达娃大[机器学习]'],"['daw[GOOD]', 'dawcdfgfrg[SKILLED]']",['c[COMMONLY]'],['15890344031[泰迪科技实习证明]']


In [4]:
for hunter_sex in frame_data['hunter_sex'].unique():
    if pd.isna(hunter_sex): continue
    print(hunter_sex, frame_data[frame_data['hunter_sex'] == hunter_sex].iloc[0, 0])

hunter_sex_kind_json = {0: '男', 1: '女'}

1.0 1649711597649788928
0.0 1649221278801985536


In [5]:
for exp_require_kind in frame_data['exp_require_kind'].unique():
    if pd.isna(exp_require_kind): continue
    print(exp_require_kind, frame_data[frame_data['exp_require_kind'] == exp_require_kind].iloc[0, 0])

exp_require_json = {0: '实习', 1: '无', 2: '全职'}

0.0 1649711597649788928
2.0 1649221278801985536
1.0 1484409030448381952


In [6]:
for hunter_sociality in frame_data['hunter_soci'].unique():
    if pd.isna(hunter_sociality): continue
    print(hunter_sociality, frame_data[frame_data['hunter_soci'] == hunter_sociality].iloc[0, 0])

require_kind_json = {'MASSES': '群众', 'LEAGUE_MEMBER': '共青团员', 
                     'PARTY_MEMBER': '党员', 'PROBATIONARY_PARTY_MEMBER': 
                     '预备党员', 'OTHER_PARTY': '其他党派'}

LEAGUE_MEMBER 1649711597649788928
OTHER_PARTY 1648848763151843328
MASSES 1648774046462115840
PARTY_MEMBER 1648779874627878912
PROBATIONARY_PARTY_MEMBER 1648621300526219264


In [7]:
save_frame = {"序号": [], '求职者ID': [], '姓名': [], '性别': [], '年龄': [], '出生地': [], '政治面貌': [], '工作经验': [], 
              '期望岗位': [], '期望薪资': [], '期望类型': [], '期望城市': [], '期望行业': [], '工作经历': [], 
              '项目经历': [], '竞赛经历': [], '教育经历': [], '培训经历': [], '技能类型': [], '语言类型': [], '证书类型': []}

TEST, RECORD, count = False, 5, 0
size = frame_data.shape[0]

for inx in range(size):
    if TEST and inx % RECORD == 0: continue
    item = frame_data.iloc[inx, :]
    count += 1
    save_frame['序号'].append(count)
    save_frame['求职者ID'].append('\t' + str(item['hunter_id']))
    save_frame['姓名'].append(item['hunter_name'])
    save_frame['性别'].append(hunter_sex_kind_json[item['hunter_sex']] if not pd.isna(item['hunter_sex']) else '')
    try:
        age = item['hunter_bthday'].replace('-', '/').replace('T', ' ').split(' ')[0]
        begin_year = datetime.strptime(age, '%Y/%m/%d').year
        end_year = datetime.now().year
        age = end_year - begin_year
    except:
        age = ''
    save_frame['年龄'].append(age)
    try:
        addr = cpca.transform([item['hunter_addr']])
        addr = ''.join([addr['省'][0], addr['市'][0]]).replace('市县', '市')
    except:
        addr = ''
    save_frame['出生地'].append(addr)
    save_frame['政治面貌'].append(require_kind_json[item['hunter_soci']] if not pd.isna(item['hunter_soci']) else '')
    try:
        work_year = '无' if '无经验' in item['hunter_exp'] else (
            item['hunter_exp'].replace('工作经验', '') if '工作经验' in item['hunter_exp'] else item['hunter_exp'])
    except:
        work_year = ''
    save_frame['工作经验'].append(work_year)
    # item['exp_position'] = re.sub(r'\\', '', item['exp_position']).strip("\"")
    save_frame['期望岗位'].append(re.sub(r'\\', '', item['exp_position']).strip("\""))
    save_frame['期望薪资'].append([item['exp_min_wage'], item['exp_max_wage']])
    save_frame['期望类型'].append(exp_require_json[item['exp_require_kind']] if not pd.isna(item['exp_require_kind']) else '')
    # save_frame['期望城市'].append(item['exp_city'])
    exp_city = re.sub(r'\\', '', str(item['exp_city'])).strip("\"")
    try:
        exp_city = ''.join(eval(exp_city))
    except:
        if not isinstance(exp_city, str) or exp_city == 'nan': exp_city = ''
    save_frame['期望城市'].append(exp_city)
    # save_frame['期望行业'].append(item['exp_industry'])
    save_frame['期望行业'].append(re.sub(r'\\', '', item['exp_industry']).strip("\""))
    # save_frame['预计到岗'].append(item['exp_report'].replace('到岗', '') if not pd.isna(item['exp_report']) else '')
    save_frame['工作经历'].append([exp.split(':')[0] for exp in eval(item['job_exps'])])
    save_frame['项目经历'].append([exp.split(':')[0] for exp in eval(item['project_exps'])])
    save_frame['竞赛经历'].append([exp.split(':')[0] for exp in eval(item['competition_exps'])])
    save_frame['教育经历'].append([exp.split(':')[0] for exp in eval(item['education_exps'])])
    save_frame['培训经历'].append(eval(item['training_exps']))
    save_frame['技能类型'].append(eval(item['skill_exps']))
    save_frame['语言类型'].append(eval(item['language_exps']))
    save_frame['证书类型'].append(eval(item['cert_exps']))

print(size, len(save_frame['序号']))
# for key, value in save_frame.items():
#     print(key, len(value))

save_frame = pd.DataFrame(save_frame)
save_frame.to_csv(save_csv_path, index=False, encoding="GBK", errors='ignore')

10902 10902
